In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot


In [2]:
df_train = pd.read_csv("TA_Training_Set.csv")
df_test = pd.read_csv("TestFileTemplate.csv")

In [3]:
df_train=df_train.sample(n=100000,random_state=42)

In [4]:
df_test=df_test.loc[df_test['Comment'].str.contains(r'[^\x00-\x7F]+') == False]
df_train=df_train.loc[df_train['Comment'].str.contains(r'[^\x00-\x7F]+') == False]
df_test

,Comment
0,"I mean the white walkers, not the wights. Was ..."
1,If that's your standard for spoilers I hope y...
2,That was a big reason why a lot used the Glitc...
3,Middleton probably does have green sweat consi...
4,Wow someone posted detected repost and linked ...
5,Did we forget that hes a flaming pile of shit?...
7,"I can't, not again. I'm not strong enough."
8,Wooow you are so pretty
9,TIL I am a very passionate person.
10,I and a few other guys had a bottle of whiskey...


In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

df_train['x_without_stopwords'] = df_train['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


df_test['x_without_stopwords'] = df_test['Comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\nÄô"
for i in symbols:
    df_train["x_without_stopwords"] = df_train["x_without_stopwords"].str.replace(i,'')
    df_train["x_without_stopwords"] = df_train["x_without_stopwords"].str.replace('\d+', '')
    df_test["x_without_stopwords"] = df_test["x_without_stopwords"].str.replace(i,'')
    df_test["x_without_stopwords"] = df_test["x_without_stopwords"].str.replace('\d+', '')

<ipython-input-6-c1b26f9c7b70>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train["x_without_stopwords"] = df_train["x_without_stopwords"].str.replace('\d+', '')
<ipython-input-6-c1b26f9c7b70>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test["x_without_stopwords"] = df_test["x_without_stopwords"].str.replace('\d+', '')
<ipython-input-6-c1b26f9c7b70>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_train["x_without_stopwords"] = df_train["x_without_stopwords"].str.replace(i,'')
<ipython-input-6-c1b26f9c7b70>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_tes

In [7]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(w) for w in text]

df_train['x_without_stopwords'].apply(lemmatize_text)

df_train["tokenized"]=df_train['x_without_stopwords'].apply(word_tokenize)


df_test['x_without_stopwords'].apply(lemmatize_text)

df_test["tokenized"]=df_test['x_without_stopwords'].apply(word_tokenize)

df_train.head(5)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shara\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Comment,Topic,x_without_stopwords,tokenized
527984,I'm destroying kids online with kitana. Underr...,7,I'm destroying kids online kitana Underrated imo,"[I, 'm, destroying, kids, online, kitana, Unde..."
618093,My guess : Not Riot job to work on this issue ...,23,My guess Not Riot job work issue Russia proba...,"[My, guess, Not, Riot, job, work, issue, Russi..."
484217,Hemsworth bending the knee in the early photo ...,24,Hemsworth bending knee early photo either 'cau...,"[Hemsworth, bending, knee, early, photo, eithe..."
161123,Oh that's more appropriate...since I did eats ...,30,Oh that's appropriatesince I eats,"[Oh, that, 's, appropriatesince, I, eats]"
234835,The fact that there are multiple points of act...,26,The fact multiple points action flow one anoth...,"[The, fact, multiple, points, action, flow, on..."


In [8]:
df_train['Topic'].unique()

array([ 7, 23, 24, 30, 26, 25,  1, 35, 16, 31, 36,  2,  5, 27, 11,  9, 20,
       38, 18, 34,  4, 33, 19, 17, 12, 21,  3, 15, 10, 13, 40, 22, 14, 32,
       39,  6, 28, 29, 37,  8], dtype=int64)

In [9]:
from matplotlib import pyplot as plt
#from sklearn.datasets.samples_generator import make_blobs
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [10]:
X= df_train['tokenized'].apply(str)
y =df_train['Topic']


X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.3,random_state=8)



print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

Train data len:57502
Class distributionCounter({20: 1612, 30: 1584, 29: 1576, 16: 1568, 25: 1568, 18: 1560, 33: 1559, 35: 1553, 2: 1552, 23: 1547, 21: 1541, 4: 1536, 10: 1535, 14: 1528, 12: 1517, 26: 1499, 11: 1498, 38: 1498, 40: 1494, 28: 1464, 39: 1457, 31: 1456, 22: 1449, 7: 1448, 3: 1446, 27: 1443, 9: 1441, 15: 1438, 37: 1430, 17: 1425, 19: 1419, 36: 1401, 5: 1398, 24: 1387, 13: 1386, 34: 1364, 1: 1319, 32: 1296, 6: 1276, 8: 34})
Valid data len:24645
Class distributionCounter({23: 753, 26: 702, 16: 687, 12: 680, 22: 678, 9: 671, 30: 668, 4: 668, 20: 663, 33: 658, 18: 647, 39: 646, 21: 646, 35: 645, 2: 645, 40: 639, 15: 638, 14: 637, 25: 637, 28: 634, 34: 632, 19: 628, 5: 626, 13: 623, 27: 623, 17: 622, 10: 620, 38: 620, 31: 615, 29: 609, 7: 602, 36: 601, 3: 593, 24: 589, 11: 586, 1: 572, 32: 560, 37: 549, 6: 522, 8: 11})


In [11]:
num_words = 50000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(X_train)

In [12]:
x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(df_train["x_without_stopwords"].tolist()) )



x_train = pad_sequences(x_train, padding='post', maxlen=100)
x_valid = pad_sequences(x_valid, padding='post', maxlen=100)
x_test = pad_sequences(x_test, padding='post', maxlen=100)



train_labels = np.asarray(y_train )
valid_labels = np.asarray( y_valid)

test_labels = np.asarray(df_train["Topic"].tolist())

print('Train data len:'+str(len(x_train)))
print('Class distribution'+str(Counter(train_labels)))

print('Validation data len:'+str(len(x_valid)))
print('Class distribution'+str(Counter(valid_labels)))

print('Test data len:'+str(len(x_test)))
print('Class distribution'+str(Counter(test_labels)))


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))


<ipython-input-12-a62070985974>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array( tokenizer.texts_to_sequences(X_train) )
<ipython-input-12-a62070985974>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
<ipython-input-12-a62070985974>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' 

Train data len:57502
Class distributionCounter({20: 1612, 30: 1584, 29: 1576, 16: 1568, 25: 1568, 18: 1560, 33: 1559, 35: 1553, 2: 1552, 23: 1547, 21: 1541, 4: 1536, 10: 1535, 14: 1528, 12: 1517, 26: 1499, 11: 1498, 38: 1498, 40: 1494, 28: 1464, 39: 1457, 31: 1456, 22: 1449, 7: 1448, 3: 1446, 27: 1443, 9: 1441, 15: 1438, 37: 1430, 17: 1425, 19: 1419, 36: 1401, 5: 1398, 24: 1387, 13: 1386, 34: 1364, 1: 1319, 32: 1296, 6: 1276, 8: 34})
Validation data len:24645
Class distributionCounter({23: 753, 26: 702, 16: 687, 12: 680, 22: 678, 9: 671, 30: 668, 4: 668, 20: 663, 33: 658, 18: 647, 39: 646, 21: 646, 35: 645, 2: 645, 40: 639, 15: 638, 14: 637, 25: 637, 28: 634, 34: 632, 19: 628, 5: 626, 13: 623, 27: 623, 17: 622, 10: 620, 38: 620, 31: 615, 29: 609, 7: 602, 36: 601, 3: 593, 24: 589, 11: 586, 1: 572, 32: 560, 37: 549, 6: 522, 8: 11})
Test data len:82147
Class distributionCounter({23: 2300, 20: 2275, 16: 2255, 30: 2252, 33: 2217, 18: 2207, 25: 2205, 4: 2204, 26: 2201, 35: 2198, 2: 2197, 12:

In [13]:
max_features =50000
embedding_dim =16
sequence_length = 100

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                    embeddings_regularizer = regularizers.l2(0.005))) 
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.LSTM(embedding_dim,dropout=0.2, recurrent_dropout=0.2,return_sequences=True,\
                                                             kernel_regularizer=regularizers.l2(0.005),\
                                                             bias_regularizer=regularizers.l2(0.005)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Dense(8, activation='relu',\
                                kernel_regularizer=regularizers.l2(0.001),\
                                bias_regularizer=regularizers.l2(0.001),))
model.add(tf.keras.layers.Dropout(0.4))


model.add(tf.keras.layers.Dense(1,activation='softmax'))

model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),optimizer=tf.keras.optimizers.Adam(1e-3),metrics=[tf.keras.metrics.BinaryAccuracy()])
                               

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           800016    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 lstm (LSTM)                 (None, 100, 16)           2112      
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 512)               819712    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 8)                 4

In [45]:
epochs = 20
# Fit the model using the train and test datasets.
#history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
history = model.fit(train_ds.shuffle(5000).batch(1024),
                    epochs= epochs ,
                    validation_data=valid_ds.batch(1024),
                    verbose=1)

Epoch 1/20
57/57 [==============================] - 15s 231ms/step - loss: -11410.8896 - binary_accuracy: 0.0229 - val_loss: -60174.4961 - val_binary_accuracy: 0.0232
Epoch 2/20
57/57 [==============================] - 15s 273ms/step - loss: -239565.3750 - binary_accuracy: 0.0229 - val_loss: -545309.0000 - val_binary_accuracy: 0.0232
Epoch 3/20
57/57 [==============================] - 16s 279ms/step - loss: -1069780.7500 - binary_accuracy: 0.0229 - val_loss: -1798874.0000 - val_binary_accuracy: 0.0232
Epoch 4/20
57/57 [==============================] - 17s 307ms/step - loss: -2830746.2500 - binary_accuracy: 0.0229 - val_loss: -4135700.7500 - val_binary_accuracy: 0.0232
Epoch 5/20
57/57 [==============================] - 17s 304ms/step - loss: -5811840.0000 - binary_accuracy: 0.0229 - val_loss: -7877214.0000 - val_binary_accuracy: 0.0232
Epoch 6/20
57/57 [==============================] - 17s 296ms/step - loss: -10392667.0000 - binary_accuracy: 0.0229 - val_loss: -13346843.0000 - val_bi